<a href="https://colab.research.google.com/github/okwasna/dane_bez_usuwania/blob/main/dane_bez_usuwania.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas


In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score


In [3]:
# Wczytanie danych z pliku Excel
file_path = 'summary_htseq_norm2.xlsx'
df = pd.read_excel(file_path, sheet_name='summary_htseq')
# Wyświetlenie liczby wszystkich genów przed filtrowaniem
print(f"Liczba genów przed filtrowaniem: {len(df)}")

# Wyświetlenie liczby wszystkich genów przed filtrowaniem
print(f"Liczba genów przed filtrowaniem: {len(df)}")


Liczba genów przed filtrowaniem: 28677
Liczba genów przed filtrowaniem: 28677


In [5]:

import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
# Wybór kolumn zaczynających się od 'SRR7451446'
# Zakładamy, że 'SRR7451446' to nazwa kolumny, a próbki znajdują się w kolumnach
data_values = df.loc[:, 'SRR7451446':]  # Wybieramy wszystkie kolumny od 'SRR7451446'

# Wyświetlenie liczby próbek (kolumn) i liczby genów (wierszy) branych pod uwagę
print(f"Liczba próbek (kolumn) branych pod uwagę: {data_values.shape[1]}")
print(f"Liczba genów (wierszy) branych pod uwagę po wybraniu próbek: {data_values.shape[0]}")

# Normalizacja danych (wierszy - genów)
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data_values)

### 1. KMeans dla 40 klastrów
n_clusters = 40
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(data_normalized)

# Dodanie kolumny z wynikami KMeans do ramki danych
df.loc[:, f'Cluster_{n_clusters}'] = kmeans.labels_

# Obliczenie wskaźnika Silhouette dla KMeans
silhouette_avg = silhouette_score(data_normalized, kmeans.labels_)
print(f"Średni wskaźnik Silhouette dla KMeans (40 klastrów): {silhouette_avg:.4f}")

# Zliczenie ilości genów w każdym klastrze
cluster_counts = df[f'Cluster_{n_clusters}'].value_counts()
print(f"Ilość genów w każdym klastrze dla KMeans (40 klastrów):")
print(cluster_counts)

Liczba próbek (kolumn) branych pod uwagę: 26
Liczba genów (wierszy) branych pod uwagę po wybraniu próbek: 28677
Średni wskaźnik Silhouette dla KMeans (40 klastrów): 0.7343
Ilość genów w każdym klastrze dla KMeans (40 klastrów):
Cluster_40
14    22212
0      3533
33     1633
20      626
7       270
23      151
37       55
29       53
39       47
17       33
35        9
16        7
31        6
5         4
15        4
24        3
30        3
25        3
19        2
34        2
38        2
2         1
6         1
36        1
3         1
9         1
22        1
11        1
18        1
32        1
21        1
28        1
1         1
4         1
26        1
10        1
13        1
12        1
8         1
27        1
Name: count, dtype: int64


In [6]:

### 2. Agglomerative Clustering dla 40 klastrów
agg_clustering = AgglomerativeClustering(n_clusters=40)
agg_labels = agg_clustering.fit_predict(data_normalized)

# Dodanie kolumny z wynikami Agglomerative Clustering do ramki danych
df.loc[:, 'Agg_Cluster_40'] = agg_labels

# Obliczenie wskaźnika Silhouette dla Agglomerative Clustering
agg_silhouette_avg = silhouette_score(data_normalized, agg_labels)
print(f"Średni wskaźnik Silhouette dla Agglomerative Clustering (40 klastrów): {agg_silhouette_avg:.4f}")

# Zliczenie ilości genów w każdym klastrze dla Agglomerative Clustering
agg_cluster_counts = pd.Series(agg_labels).value_counts()
print(f"Ilość genów w każdym klastrze dla Agglomerative Clustering (40 klastrów):")
print(agg_cluster_counts)



Średni wskaźnik Silhouette dla Agglomerative Clustering (40 klastrów): 0.7252
Ilość genów w każdym klastrze dla Agglomerative Clustering (40 klastrów):
12    21787
22     4004
1      1886
7       387
19      265
5       156
9        78
4        26
2        18
8         9
15        8
18        6
0         6
6         5
10        4
13        3
14        3
17        3
3         2
39        1
35        1
16        1
36        1
23        1
33        1
28        1
25        1
27        1
29        1
20        1
31        1
26        1
24        1
30        1
32        1
38        1
21        1
37        1
34        1
11        1
Name: count, dtype: int64


In [1]:
### 3. DBSCAN
# DBSCAN - nie musimy podawać liczby klastrów, ale parametry epsilon i min_samples
dbscan_clustering = DBSCAN(eps=0.5, min_samples=10)
dbscan_labels = dbscan_clustering.fit_predict(data_normalized)

# Dodanie kolumny z wynikami DBSCAN do ramki danych
df.loc[:, 'DBSCAN_Cluster'] = dbscan_labels

# Zliczenie ilości genów w każdym klastrze dla DBSCAN
dbscan_cluster_counts = pd.Series(dbscan_labels).value_counts()
print(f"Ilość genów w każdym klastrze dla DBSCAN:")
print(dbscan_cluster_counts)

# Obliczenie wskaźnika Silhouette dla DBSCAN (jeśli są więcej niż 1 klaster i nie są outliery)
if len(set(dbscan_labels)) > 1 and -1 not in dbscan_labels:
    dbscan_silhouette_avg = silhouette_score(data_normalized, dbscan_labels)
    print(f"Średni wskaźnik Silhouette dla DBSCAN: {dbscan_silhouette_avg:.4f}")
else:
    print("Nie można obliczyć wskaźnika Silhouette dla DBSCAN (za mało klastrów lub obecność outlierów).")


NameError: name 'DBSCAN' is not defined